# Standardize Filters

* We are using lots of different filter traces, especially when we combine the UAT Groups project and Virgo.
* We have filter traces from lots of different sources.
* I want to write them out in a uniform way, with wavelength and transmission in the same units for all.
* I also want to write them in fits format so we can easily store the units and column names.

In [1]:
import os
from matplotlib import pyplot as plt
import numpy as np
from astropy.table import Table